## Baseline Estimation

In [1]:
import pandas as pd
import numpy as np
from tick_loss import *
import requests

In [2]:
url = "https://raw.githubusercontent.com/TobiPfeiffersGitHub/BSExNovartis_Thesis/main/Data/monthly_data.csv"
response = requests.get(url)

if response.status_code == 200:
    with open("monthly_data.csv", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download the file.")

# Load the file into a DataFrame
data = pd.read_csv("monthly_data.csv")

data

File downloaded successfully.


,Unnamed: 0,Date,unemployment_rate,inflation_rate,treasury_yield,Monthly Real GDP Index,GDP_growth,SP500_return,ABBV,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG
0,0,2000-01-01,4.0,169.300,NaN,12870.47162,-1.066967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2000-02-01,4.1,170.000,6.661000,12934.38121,0.496560,-1.522563,NaN,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789
2,2,2000-03-01,4.0,171.000,6.519500,13000.50928,0.511258,9.413333,NaN,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258
3,3,2000-04-01,3.8,170.900,6.256522,13173.13041,1.327803,-3.266805,NaN,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518
4,4,2000-05-01,4.0,171.200,5.990526,13149.18201,-0.181797,-1.572223,NaN,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,278,2023-03-01,3.5,301.808,3.746842,NaN,NaN,3.313488,3.554254,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813
279,279,2023-04-01,3.4,302.918,3.663043,NaN,NaN,1.985238,-5.176628,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467
280,280,2023-05-01,3.7,NaN,3.460000,NaN,NaN,0.461619,-7.868480,-0.191204,-2.695194,-5.277949,8.487855,-4.382088,-3.967915,-6.161645,-2.237080,-2.162160
281,281,2023-06-01,NaN,NaN,3.573636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data = data.drop(data.index[0])
data = data.drop(data.index[-3:])
data = data.reset_index(drop=True)
data = data.drop(columns=['Unnamed: 0'])
data

,Date,unemployment_rate,inflation_rate,treasury_yield,Monthly Real GDP Index,GDP_growth,SP500_return,ABBV,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG
0,2000-02-01,4.1,170.000,6.661000,12934.38121,0.496560,-1.522563,NaN,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789
1,2000-03-01,4.0,171.000,6.519500,13000.50928,0.511258,9.413333,NaN,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258
2,2000-04-01,3.8,170.900,6.256522,13173.13041,1.327803,-3.266805,NaN,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518
3,2000-05-01,4.0,171.200,5.990526,13149.18201,-0.181797,-1.572223,NaN,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265
4,2000-06-01,4.0,172.200,6.440455,13189.36127,0.305565,1.728613,NaN,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,2022-12-01,3.5,298.990,3.891000,20204.80519,-0.085161,-6.193593,0.266786,-0.250105,-10.376181,-0.117873,-1.137072,0.753719,8.619586,1.295221,3.090302,9.446071
275,2023-01-01,3.4,300.536,3.616190,20283.65522,0.390254,6.776820,-8.576205,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326
276,2023-02-01,3.6,301.648,3.531500,20193.96427,-0.442183,-2.514271,5.152609,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681
277,2023-03-01,3.5,301.808,3.746842,NaN,NaN,3.313488,3.554254,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813


In [4]:
cutoff = 12